In [1]:
import os
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.environment import Environment
from azureml.core import ScriptRunConfig
from azureml.core import Dataset
from azureml.data.datapath import DataPath
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.conda_dependencies import CondaDependencies

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.53.0


### Prihlasenie do azure

In [2]:
ws = Workspace.from_config('config_ns.json', auth=InteractiveLoginAuthentication())
print(ws.name, ws.location, ws.resource_group, sep='\t')

ns-fiit-2023-2-workspace	westeurope	ns-fiit-povinec


### Nacitanie datasetu

In [5]:
Dataset.get_all(workspace=ws)

{}

In [11]:
dataset = Dataset.get_by_name(workspace=ws, name='VSDataset')

Request failed with: 404 Client Error: Data version VSDataset:None (dataContainerName:version) not found. for url: https://westeurope.experiments.azureml.net/data/v1.0/subscriptions/885fd293-53ed-4e5a-8767-d6aaddbc1be5/resourceGroups/ns-fiit-povinec/providers/Microsoft.MachineLearningServices/workspaces/ns-fiit-2023-2-workspace/dataversion/VSDataset/versions/None?includeSavedDatasets=true
Tried to retrieve v2 data asset but could not find v2 data assetregistered with name "VSDataset" (version: None) in the workspace.
2023-11-07 11:30:35.079251 | ActivityCompleted: Activity=get_by_name, HowEnded=Failure, Duration=1180.5 [ms], Info = {'activity_id': '6d55f110-fc87-40e9-889d-a9be983ae9b0', 'activity_name': 'get_by_name', 'activity_type': 'PublicApi', 'app_name': 'dataset', 'source': 'azureml.dataset', 'version': '1.53.0', 'dataprepVersion': '', 'sparkVersion': '', 'subscription': '', 'run_id': '', 'resource_group': '', 'workspace_name': '', 'experiment_id': '', 'location': '', 'completion

UserErrorException: UserErrorException:
	Message: Cannot find dataset registered with name "VSDataset" (version: None) in the workspace.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Cannot find dataset registered with name \"VSDataset\" (version: None) in the workspace."
    }
}

## Experimenty

### Training

In [13]:
config = ScriptRunConfig(source_directory='../src/',
                         script='train.py',
                         arguments=[
                           #'--wandb', "a68b79c89593d1f8c50631508f2e618f3ba7ccf3",
                            '--data_path', dataset.as_mount()], # This is important how to mount dataset from DataStore
                         compute_target='K80x1-xpovinec') # Compute target is your created compute cluster


### Evaluation - normal

In [ ]:
config = ScriptRunConfig(source_directory='../src/',
                          script='evaluation/evaluation_hypo.py',
                          arguments=[
                              '--wandb', "a68b79c89593d1f8c50631508f2e618f3ba7ccf3",
                              '--data_path', dataset.as_mount(),
                              '--model_path', 'models/',
                              '--channels', 3
                          ], 
                         compute_target='K80x1-xpovinec') # Compute target is your created compute cluster


### Evaluation - separate

In [33]:
config = ScriptRunConfig(source_directory='../src/',
                          script='evaluation/evaluation_separate.py',
                          arguments=[
                              '--wandb', "a68b79c89593d1f8c50631508f2e618f3ba7ccf3",
                              '--data_path', dataset.as_mount(),
                              '--model_path', 'models/separate/2ch',
                              '--channels', 2
                          ], 
                         compute_target='K80x1-xpovinec') # Compute target is your created compute cluster


### Setting the experiment

In [14]:
experiment = Experiment(workspace=ws, name='xpovinec-test')

### Nastavenie env

In [ ]:
# toto vypise vsetky env
for i,key in enumerate(ws.environments.keys()):
    print(f"{i}.",key)

In [ ]:
ws.environments

In [19]:
# vytvorenie noveho env
conda = CondaDependencies()

# add channels
#conda.add_channel('pytorch')

# add conda packages
conda.add_conda_package('pip=20.2.4')
conda.add_conda_package('python=3.8')

# add pip packages
conda.add_pip_package('keras~=2.6.0')
#conda.add_pip_package('Keras-Preprocessing~=2.8.0')
conda.add_pip_package('matplotlib~=3.5.1')
conda.add_pip_package('nibabel~=3.2.2')
conda.add_pip_package('numpy~=1.21.3')
conda.add_pip_package('pandas~=1.4.1')
conda.add_pip_package('scikit-learn~=1.0.2')
conda.add_pip_package('scipy~=1.7.0')
conda.add_pip_package('tensorflow-gpu~=2.7.0')
conda.add_pip_package('azureml-core~=1.39.0')
conda.add_pip_package('opencv-python~=4.5.5.64')
conda.add_pip_package('opencv-python-headless~=4.5.5.64')
conda.add_pip_package('scikit-image')
conda.add_pip_package('segmentation-models-3D')

# create environment
env = Environment('xpovinec-env')
env.python.conda_dependencies = conda
env.save_to_directory('../', overwrite=True)

In [ ]:
env

In [62]:
# vytvorenie noveho env z requirements.txt
#env = Environment.from_pip_requirements(name='xpovinec-env', file_path='../requirements.txt')

In [ ]:
# pridanie na azure, ked zmenim nejaky package tak sa updatne
env.register(workspace=ws)

In [18]:
# nieco s dockerom, neviem ci to treba davat vzdy
azureml.core.runconfig.DockerConfiguration(use_docker=True)

In [15]:
# vyber env
env = Environment.get(workspace=ws, name='xpovinec-env')
#env

In [16]:
# nastavenie env pre ten experiment
config.run_config.environment = env

### Spustenie experimentu

In [17]:
run = experiment.submit(config)

In [10]:
# toto zisti, kde bezi a vrati to stranku
aml_url = run.get_portal_url()
print(aml_url)

https://ml.azure.com/runs/xpovinec-test_1651557556_0e831090?wsid=/subscriptions/bc82dccd-f19d-42cb-9ce3-0d5df33ef086/resourcegroups/a0047-STUFIIT-ML01/workspaces/a0047stufiitml01&tid=5dbf1add-202a-4b8d-815b-bf0fb024e033
